In [1]:
# load -> clean -> features -> fit/validate -> predict -> submit
from linear_model_base import RidgeRegression
import numpy as np

In [2]:
from data_cleaner import Data_Cleaner

#80/20 train test split to verify predictions before upload

data_cleaner_train = Data_Cleaner("C:/Users/Tim/Documents/GitHub/MLProject1/project1/data/train.csv")
data_cleaner_train.tX = data_cleaner_train.tX[:200000,:]
data_cleaner_train.y = data_cleaner_train.y[:200000]

data_cleaner_train._fill_with_NaN()
data_cleaner_train.replace_with_zero()

data_cleaner_test = Data_Cleaner("C:/Users/Tim/Documents/GitHub/MLProject1/project1/data/train.csv")
data_cleaner_test.tX = data_cleaner_test.tX[200000:,:]
data_cleaner_test.y = data_cleaner_test.y[200000:]

data_cleaner_test._fill_with_NaN()
data_cleaner_test.replace_with_zero()

In [4]:
def poly_buildup(x):
    """add features with 1 increased polynomial order to featureset containing 30 features"""
    current_degree = (x.shape[1])/30
    return np.hstack([x,x[:,0:30]**(current_degree+1)])

def build_poly(x, degree, add_degree_zero=False):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # polynomial basis function: TODO
    # this function should return the matrix formed
    # by applying the polynomial basis to the input data
    if add_degree_zero:
        xN = np.hstack([np.ones([x.shape[0],1]),x])
    else:
        xN = x
    if degree>0:
        for i in range(degree-1):
            xN = np.hstack([xN, x**(i+2)])
    return np.array(xN)

In [5]:
from linear_model_base import RidgeRegression
import numpy as np 

min_rmse_te = []
min_rmse_tr = []
lambdas = []
min_max = []

for poly_order in range(5):
       
    #iteratively increase polynomial features     
    if poly_order > 0:
        #re-scale back to original scale, standardize polynomial features after they are generated
        data_cleaner_train.tX = data_cleaner_train.tX*(maximum-minimum)+minimum
        data_cleaner_train.tX = poly_buildup(data_cleaner_train.tX)
    
    #standardize
    minimum, maximum = data_cleaner_train.getMinMax()
    min_max.append(data_cleaner_train.getMinMax())
    data_cleaner_train.standardize()
    
    Model = RidgeRegression(data_cleaner_train)
    search_space = np.logspace(-10, 0, 10)
    
    rmse_te = []
    rmse_tr = []
    
    for lambda_ in search_space:
        
        tr, te = Model.cross_validation(5, lambda_=lambda_)
        rmse_te.append(te)
        rmse_tr.append(tr)
        
        print("lambda {}".format(lambda_))
        
    min_rmse_te.append(np.min(rmse_te))
    min_rmse_tr.append(np.min(rmse_tr))
    lambdas.append(search_space[np.where(rmse_te==np.min(rmse_te))])
        

lambda 1e-10
lambda 1.2915496650148826e-09
lambda 1.6681005372000592e-08
lambda 2.1544346900318867e-07
lambda 2.782559402207126e-06
lambda 3.5938136638046256e-05
lambda 0.0004641588833612782
lambda 0.005994842503189421
lambda 0.07742636826811278
lambda 1.0
lambda 1e-10
lambda 1.2915496650148826e-09
lambda 1.6681005372000592e-08
lambda 2.1544346900318867e-07
lambda 2.782559402207126e-06
lambda 3.5938136638046256e-05
lambda 0.0004641588833612782
lambda 0.005994842503189421
lambda 0.07742636826811278
lambda 1.0
lambda 1e-10
lambda 1.2915496650148826e-09
lambda 1.6681005372000592e-08
lambda 2.1544346900318867e-07
lambda 2.782559402207126e-06
lambda 3.5938136638046256e-05
lambda 0.0004641588833612782
lambda 0.005994842503189421
lambda 0.07742636826811278
lambda 1.0
lambda 1e-10
lambda 1.2915496650148826e-09
lambda 1.6681005372000592e-08
lambda 2.1544346900318867e-07
lambda 2.782559402207126e-06
lambda 3.5938136638046256e-05
lambda 0.0004641588833612782
lambda 0.005994842503189421
lambda 0.0

In [7]:
#find optimal highest polynomial order, lambda and scale factors for test set
n_poly = np.where(min_rmse_te==np.min(min_rmse_te))[0]
best_lambda = lambdas[int(n_poly)][0]
minimum, maximum = min_max[int(n_poly)]

In [8]:
weights = Model._run(lambda_ = best_lambda)

In [9]:
from proj1_helpers import *
#generate poly features and standardise according to min_max from training
data_cleaner_test.tX = build_poly(data_cleaner_test.tX, int(n_poly+1))
data_cleaner_test.tX = (data_cleaner_test.tX - minimum) / (maximum - minimum)

In [10]:
Model = RidgeRegression(data_cleaner_test)
weights = Model._run(lambda_ = best_lambda)

y_pred = predict_labels(weights, data_cleaner_test.tX)

In [11]:
#estimate accuracy based on test set
def estimate_Leaderboard_score(y_true,weights,data):
    """Helper function estimating the categorical accuracy on the leaderscore
    """
    y_pred = predict_labels(weights, data)
    N_tot = y_pred.shape[0]
    N_true = len(np.where(y_pred == y_true)[0])
    categorical_acuracy = N_true/N_tot
    return categorical_acuracy

estimate_Leaderboard_score(data_cleaner_test.y, weights, data_cleaner_test.tX)

0.78508

In [12]:
#generate predictions
data_cleaner_upload = Data_Cleaner("C:/Users/Tim/Documents/GitHub/MLProject1/project1/data/test.csv")
data_cleaner_upload._fill_with_NaN()
data_cleaner_upload.replace_with_zero()
data_cleaner_upload.tX = build_poly(data_cleaner_upload.tX, int(n_poly+1))
data_cleaner_upload.tX = (data_cleaner_upload.tX - minimum) / (maximum - minimum)

y_pred = predict_labels(weights, data_cleaner_upload.tX)
create_csv_submission(data_cleaner_upload.ids, y_pred, "C:/Users/Tim/Documents/GitHub/MLProject1/project1/data/RidgePoly3.csv")